## Extraktionsbaserad textsammanfattare med olika rankningsmått 

Prio: 
* Hämtning | vi väljer korpus 
* Cleaning / preprocessing 
* (Under tiden fundera på om vi skulle kunna använda maskinlärningsmetoder på något sätt)

# ALGORITHM 
### Preprocessing
* Input corpus 
* Store away stopwords 

### Rankingmått 
* Headings
* Cohesion and lexical connectedness criteria
Finns underkriterier för måttet 
* NER (nltk lib)

* Store score for each sentence 

### Assemble output 
* Reassemble according to ranking 
* Output summarization 

In [15]:
# weighted frequency = (word frequency / highest word frequency)
# limit sentence length in summary? 

In [16]:
# Sentece tokenizer
# Word Tokenizer 
# Regex Tokenizer 

In [17]:
import nltk 
from urllib import request
import re
# Beautifulsoup 

In [18]:
url = "https://www.aftonbladet.se/nyheter/kolumnister/a/x8w2ln/stockholmare-ar-dummast-i-sverige"
response = request.urlopen(url)
raw = response.read().decode('utf8')
#print(type(raw))
#print(raw)
#str = raw.read()
#print(type())
tokens = nltk.word_tokenize(raw)
text = nltk.Text(tokens)

In [19]:
#paragraphs = re.compile("r\\b<p class=\"borderColor borderWidth margin padding mqDark hyperion-css-1nrt0vq\">.*</p>\\b").findall(raw)
paragraphs = re.compile("\\b<script>\\b").findall(raw)

para = re.compile("<p>[^<]*<\/p>").findall(raw)
#para = re.compile("\\b<script>.*?\\b").findall(raw)
print(type(raw))
#print(len(paragraphs))
print(len(para))

<class 'str'>
0


# Webscraping 
* Beautiful soup + urllib.request + re 

In [20]:
# pip install newspaper3k --> module för att ladda ner artiklar 

# NLTK ContextIndex --> Sliding window 

def word_similarity_dict(self, word) -> Sim scores  

Orden används i liknande kontexter --> Borde kunna ta bort redundans! 
Kan finnas andra NLTK-klasser som är användbara 
